In [1]:
import numpy as np
import pandas as pd

from src.definitions import ROOT_DIR, KEYS_TO_ORDINAL
from src.features.build_features import replace_nan_inf, shift_concat, gradient, shift_concat_gradient
from src.features.build_features import build_encoding_map, label_encode_columns

In [2]:
%load_ext autoreload
%autoreload 2

# Olawale preprocess

The competition winner employs the following steps to preprocess the data:


1. Grab target and map it to scoring matrix index (lithology_ordinal).

2. Drop columns with uncommon logs.

3. Encode Group, Formation, and Well names. In my opinion, we should drop the well names before training.

4. Fill mising values with -999. I think XGBoost can handle missing values. Alternatively, we could use a fancier imputation method.

5. Drop target column.

6. Augment features usign Bestagini's functions.

7. Return augmented features and lithology ordinal

In this notebook I explore each step, before wrinting the resulting strategy as a `preprocess` method in the `Model` class. Also, Olawale preprocessed the train and test data in the same method. I'd like to create the preprocess method for the train set, and then apply it to both train and test sets.

# Load train data

In [3]:
train_path = ROOT_DIR / 'data/external' / 'CSV_train.csv'

assert train_path.is_file()

In [4]:
df = pd.read_csv(train_path, sep=';')

In [5]:
df.sample(10)

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
1074686,35/3-7 S,3814.571851,543933.87500,6856645.5,-3709.382568,CROMER KNOLL GP.,Agat Fm.,8.549600,NaN,4.327357,...,16.797989,152.291809,NaN,-0.008794,NaN,4.705602,12.884252,NaN,65000,1.0
156992,16/5-3,1596.238000,474410.84375,6510408.5,-1574.205444,SHETLAND GP.,Tor Fm.,8.638204,NaN,2.439296,...,15.485083,146.295517,NaN,-0.002427,NaN,2.923528,15.986755,NaN,70000,1.0
626092,31/3-3,1136.522000,545898.43750,6744773.0,-1109.806152,ROGALAND GP.,Lista Fm.,NaN,0.750703,0.750504,...,NaN,NaN,NaN,0.073168,NaN,NaN,NaN,NaN,65000,1.0
901358,34/7-20,1573.430000,448295.50000,6812831.5,-1547.393188,HORDALAND GP.,Utsira Fm.,17.835421,0.712647,0.461698,...,NaN,NaN,NaN,-0.004028,NaN,NaN,NaN,NaN,65000,1.0
1039714,35/11-6,3194.847600,524979.31250,6784904.5,-3166.633545,VIKING GP.,Sognefjord Fm.,12.015563,6.954322,5.671699,...,NaN,148.631500,NaN,-0.008055,NaN,NaN,NaN,NaN,65030,1.0
851115,34/2-4,949.937436,482074.93750,6847169.5,-916.910522,NORDLAND GP.,NaN,19.033978,NaN,2.528447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65030,1.0
238178,25/2-13 T4,2524.280000,469385.03125,6628712.5,-2500.030518,ROGALAND GP.,Lista Fm.,13.239440,1.444163,1.317648,...,NaN,NaN,0.989437,0.042648,NaN,NaN,NaN,NaN,65000,1.0
778411,34/10-21,3040.034403,454127.40625,6777267.5,-3010.533936,VIKING GP.,Draupne Fm.,13.007800,3.073704,2.349300,...,NaN,NaN,NaN,-0.006764,NaN,NaN,NaN,2.978325,65000,1.0
512017,30/3-3,3217.046000,497357.75000,6737331.0,-3194.683105,DUNLIN GP.,Drake Fm.,9.109400,7.536695,6.039466,...,NaN,NaN,NaN,-0.009431,NaN,NaN,NaN,7.530255,65000,1.0
932369,34/8-1,2133.585023,469622.81250,6803746.0,-2106.637207,SHETLAND GP.,Lista Fm.,13.226599,NaN,1.681029,...,NaN,NaN,NaN,-0.007416,NaN,NaN,NaN,NaN,65000,1.0


In [6]:
df.shape

(1170511, 29)

## Raw features

In [7]:
df.columns

Index(['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'GROUP', 'FORMATION',
       'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR', 'NPHI', 'PEF',
       'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC',
       'ROPA', 'RXO', 'FORCE_2020_LITHOFACIES_LITHOLOGY',
       'FORCE_2020_LITHOFACIES_CONFIDENCE'],
      dtype='object')

# Grab target and map it to scoring matrix index

In [8]:
lith_codes = df['FORCE_2020_LITHOFACIES_LITHOLOGY']

In [9]:
lith_ordinal = lith_codes.map(KEYS_TO_ORDINAL)

# Drop target column
This should be done outside the preprocess method. This way, we can preprocess both train and test set with the same method.

In [10]:
df.drop('FORCE_2020_LITHOFACIES_LITHOLOGY', axis=1, inplace=True)

# Drop columns with uncommon logs

In [11]:
missing_values = (df.isna().sum() / df.shape[0]).sort_values(ascending=False)

print('Percent of missing values')
missing_values

Percent of missing values


SGR                                  0.940750
DTS                                  0.850823
RMIC                                 0.849502
ROPA                                 0.835691
DCAL                                 0.744699
MUDWEIGHT                            0.729903
RXO                                  0.720270
ROP                                  0.542874
RSHA                                 0.461218
PEF                                  0.426155
BS                                   0.416787
NPHI                                 0.346090
SP                                   0.261650
DRHO                                 0.156046
RHOB                                 0.137777
FORMATION                            0.117038
CALI                                 0.075076
DTC                                  0.069084
RMED                                 0.033313
RDEP                                 0.009410
Z_LOC                                0.009205
Y_LOC                             

From this list we can see that there are a few logs with high rate of missing values. Let's drop those with more than 50% missing values.  

In [12]:
missing_value_cutoff = 0.5
cols = missing_values[missing_values > missing_value_cutoff].index.to_list()
cols

['SGR', 'DTS', 'RMIC', 'ROPA', 'DCAL', 'MUDWEIGHT', 'RXO', 'ROP']

The drop columns in the competition winner code are:

| Drop column |
| ----------- |
|FORCE_2020_LITHOFACIES_CONFIDENCE|
|SGR|
|DTS|
|RXO|
|ROPA|

There is some overlap with the columns found using the 50% cutoff on the missing values. The main discrepancy is the column `FORCE_2020_LITHOFACIES_CONFIDENCE`. This column is meant to assing a certainty value (ordinal) to the lithology interpretation. However, it will not be present in the prediction data (test) and thus it can't be used directly in the training. I'll drop it here, but it will be interesting to explore how to use it during training in future work.

In [13]:
cols.append('FORCE_2020_LITHOFACIES_CONFIDENCE')

In [14]:
df.drop(cols, axis=1, inplace=True)

In [15]:
df.shape

(1170511, 19)

# Encode categorical columns

In [16]:
df.dtypes

WELL          object
DEPTH_MD     float64
X_LOC        float64
Y_LOC        float64
Z_LOC        float64
GROUP         object
FORMATION     object
CALI         float64
RSHA         float64
RMED         float64
RDEP         float64
RHOB         float64
GR           float64
NPHI         float64
PEF          float64
DTC          float64
SP           float64
BS           float64
DRHO         float64
dtype: object

In [17]:
cat_columns = df.select_dtypes(include='object').columns
print(cat_columns)

Index(['WELL', 'GROUP', 'FORMATION'], dtype='object')


The well name column `WELL` is important for referencing the sample but should not carry information about the lithologies it contains. Let's not encode it.

Also, the `GROUP` and `FORMATION` columns could carry a lot of information about the target lithology, however, including these in the model could make it less robust for application in areas where these formations and groups don't exists (e.g. a different basin) or are named differently. In this case, I will encode them trying to stay close to Olawale's work, but in the future I'd like to test the effect of droppoing these columns in the model's score.

In [18]:
cat_columns = cat_columns.drop('WELL')
print(cat_columns)

Index(['GROUP', 'FORMATION'], dtype='object')


In [19]:
df_cat = df.loc[:, cat_columns]
df_cat.head()

,GROUP,FORMATION
0,NORDLAND GP.,NaN
1,NORDLAND GP.,NaN
2,NORDLAND GP.,NaN
3,NORDLAND GP.,NaN
4,NORDLAND GP.,NaN


## Group missing values

In [20]:
len(df_cat['GROUP'].unique())

15

In [21]:
df_cat['GROUP'].value_counts(normalize=True, dropna=False)

HORDALAND GP.       0.250450
SHETLAND GP.        0.199937
VIKING GP.          0.112770
ROGALAND GP.        0.112723
DUNLIN GP.          0.101738
NORDLAND GP.        0.095249
CROMER KNOLL GP.    0.044698
BAAT GP.            0.030605
VESTLAND GP.        0.022312
HEGRE GP.           0.011886
ZECHSTEIN GP.       0.010455
BOKNFJORD GP.       0.002670
ROTLIEGENDES GP.    0.002385
NaN                 0.001092
TYNE GP.            0.001029
Name: GROUP, dtype: float64

In [22]:
group_missing_perc = df_cat['GROUP'].isna().sum() / len(df_cat['GROUP'])

print(f'The percent of missing values in GROUP is: {group_missing_perc*100:.4f}%')

The percent of missing values in GROUP is: 0.1092%


## Formation missing values

In [23]:
len(df_cat['FORMATION'].unique())

70

In [24]:
df_cat['FORMATION'].value_counts(normalize=True, dropna=False)

Utsira Fm.               0.147488
NaN                      0.117038
Kyrre Fm.                0.080587
Lista Fm.                0.060726
Heather Fm.              0.055566
                           ...   
Broom Fm.                0.000201
Intra Balder Fm. Sst.    0.000151
Farsund Fm.              0.000146
Flekkefjord Fm.          0.000101
Egersund Fm.             0.000090
Name: FORMATION, Length: 70, dtype: float64

In [25]:
formation_missing_perc = df_cat['FORMATION'].isna().sum() / len(df_cat['FORMATION'])
print(f'The percent of missing values in GROUP is: {formation_missing_perc*100:.4f}%')

The percent of missing values in GROUP is: 11.7038%


## Label encode
The competition winner uses a label encoder in pandas. Since the model is tree-based and there is high cardinality in the formation column (70 unique categories), this is a reasonable compromise. Later, we could try to use one hot encoding to test the effect in the model score.

Also, Olawale approach used -1 to mark missing values. In this case, I'll use numpy.nan instead.

In [26]:
group_mapping = build_encoding_map(df_cat['GROUP'])
group_mapping

{'NORDLAND GP.': 0,
 'HORDALAND GP.': 1,
 'ROGALAND GP.': 2,
 'SHETLAND GP.': 3,
 'CROMER KNOLL GP.': 4,
 'VIKING GP.': 5,
 'VESTLAND GP.': 6,
 'ZECHSTEIN GP.': 7,
 'HEGRE GP.': 8,
 'ROTLIEGENDES GP.': 9,
 'TYNE GP.': 10,
 'BOKNFJORD GP.': 11,
 'DUNLIN GP.': 12,
 'BAAT GP.': 13}

In [27]:
formation_mapping = build_encoding_map(df_cat['FORMATION'])
formation_mapping

{'Utsira Fm.': 1,
 'Balder Fm.': 2,
 'Sele Fm.': 3,
 'Lista Fm.': 4,
 'Heimdal Fm.': 5,
 'Tor Fm.': 6,
 'Hod Fm.': 7,
 'Blodoeks Fm.': 8,
 'Svarte Fm.': 9,
 'Roedby Fm.': 10,
 'Sola Fm.': 11,
 'Aasgard Fm.': 12,
 'Draupne Fm.': 13,
 'Heather Fm.': 14,
 'Hugin Fm.': 15,
 'Smith Bank Fm.': 16,
 'Frigg Fm.': 17,
 'Skagerrak Fm.': 18,
 'Ekofisk Fm.': 19,
 'Kupferschiefer Fm.': 20,
 'Skade Fm.': 21,
 'Grid Fm.': 22,
 'Vaale Fm.': 23,
 'Sleipner Fm.': 24,
 'Hidra Fm.': 25,
 'Tuxen Fm.': 26,
 'Mandal Fm.': 27,
 'Ula Fm.': 28,
 'Bryne Fm.': 29,
 'Tau Fm.': 30,
 'Sandnes Fm.': 31,
 'Intra Draupne Fm. Sst.': 32,
 'Statfjord Fm.': 33,
 'Skade Mb.': 34,
 'BASEMENT': 35,
 'Ran Sst Mb.': 36,
 'Flekkefjord Fm.': 37,
 'Sauda Fm.': 38,
 'Egersund Fm.': 39,
 'Intra Balder Fm. Sst.': 40,
 'Hermod Mb.': 41,
 'Ty Fm.': 42,
 'Hardraade Fm.': 43,
 'Kyrre Fm.': 44,
 'Tryggvason Fm.': 45,
 'Drake Fm.': 46,
 'Cook Fm.': 47,
 'Amundsen Fm.': 48,
 'Grid Mb.': 49,
 'Ty Mb.': 50,
 'Jorsalfare Fm.': 51,
 'Burton Fm.

In [28]:
mappings = {col: build_encoding_map(df_cat[col]) for col in df_cat}

In [29]:
df_encoded, encoded_col_names = label_encode_columns(df, cat_columns, mappings)

In [30]:
df_encoded.head()

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,NPHI,PEF,DTC,SP,BS,DRHO,GROUP_encoded,FORMATION_encoded
0,15/9-13,494.528,437641.96875,6470972.5,-469.501831,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,NaN,20.915468,161.131180,24.612379,NaN,-0.574928,0.0,NaN
1,15/9-13,494.680,437641.96875,6470972.5,-469.653809,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,NaN,19.383013,160.603470,23.895531,NaN,-0.570188,0.0,NaN
2,15/9-13,494.832,437641.96875,6470972.5,-469.805786,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,NaN,22.591518,160.173615,23.916357,NaN,-0.574245,0.0,NaN
3,15/9-13,494.984,437641.96875,6470972.5,-469.957794,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,NaN,32.191910,160.149429,23.793688,NaN,-0.586315,0.0,NaN
4,15/9-13,495.136,437641.96875,6470972.5,-470.109772,19.453100,NaN,1.602679,1.795299,1.880034,71.729141,NaN,38.495632,160.128342,24.104078,NaN,-0.597914,0.0,NaN


# Impute missing values
In the winning code, the missing values are filled with three different values:

1. -999: For missing values in the raw logs.
2. 0: This is used in Bestagini's feature augmentation to fill missing values created by these functions.
3. -1: This is use in the encoding of the categorical columns.

In my case, I'll use numpy.nan for these three cases. Since XGboost can handle missing data, I think it is an acceptable first pass.

# Augment features usign Bestagini's functions.
In notebook 4.0, I explore Bestagini's functions originally written using numpy, and compare them to my pandas version. Here, I incorporate my pandas version of these functions to the preprocess method.

In [31]:
df_preprocesed = shift_concat_gradient(df_encoded,
                                       'DEPTH_MD',
                                       'WELL',
                                       encoded_col_names,
                                       periods=1,
                                       fill_value=None)

In [32]:
df_preprocesed.head()

,X_LOC_shifted_1,Y_LOC_shifted_1,Z_LOC_shifted_1,CALI_shifted_1,RSHA_shifted_1,RMED_shifted_1,RDEP_shifted_1,RHOB_shifted_1,GR_shifted_1,NPHI_shifted_1,...,RMED_gradient,RDEP_gradient,RHOB_gradient,GR_gradient,NPHI_gradient,PEF_gradient,DTC_gradient,SP_gradient,BS_gradient,DRHO_gradient
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,437641.96875,6470972.5,-469.501831,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,NaN,...,0.043819,-0.020000,0.036893,-6.170825,NaN,-10.081944,-3.471776,-4.716108,NaN,0.031179
2,437641.96875,6470972.5,-469.653809,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,NaN,...,0.055186,0.033500,0.044271,-29.216365,NaN,21.108590,-2.827996,0.137015,NaN,-0.026689
3,437641.96875,6470972.5,-469.805786,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,NaN,...,-0.032003,0.005153,-0.030329,-12.783402,NaN,63.160470,-0.159113,-0.807034,NaN,-0.079409
4,437641.96875,6470972.5,-469.957794,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,NaN,...,-0.124441,-0.040905,-0.078150,-7.564344,NaN,41.471858,-0.138735,2.042043,NaN,-0.076305


In [33]:
df_preprocesed.shape

(1170511, 68)

In [34]:
df_preprocesed.dtypes.head(30)

X_LOC_shifted_1                float64
Y_LOC_shifted_1                float64
Z_LOC_shifted_1                float64
CALI_shifted_1                 float64
RSHA_shifted_1                 float64
RMED_shifted_1                 float64
RDEP_shifted_1                 float64
RHOB_shifted_1                 float64
GR_shifted_1                   float64
NPHI_shifted_1                 float64
PEF_shifted_1                  float64
DTC_shifted_1                  float64
SP_shifted_1                   float64
BS_shifted_1                   float64
DRHO_shifted_1                 float64
GROUP_encoded_shifted_1        float64
FORMATION_encoded_shifted_1    float64
X_LOC                          float64
Y_LOC                          float64
Z_LOC                          float64
CALI                           float64
RSHA                           float64
RMED                           float64
RDEP                           float64
RHOB                           float64
GR                       

In [35]:
df_preprocesed.dtypes.tail(38)

BS                              float64
DRHO                            float64
GROUP_encoded                   float64
FORMATION_encoded               float64
X_LOC_shifted_-1                float64
Y_LOC_shifted_-1                float64
Z_LOC_shifted_-1                float64
CALI_shifted_-1                 float64
RSHA_shifted_-1                 float64
RMED_shifted_-1                 float64
RDEP_shifted_-1                 float64
RHOB_shifted_-1                 float64
GR_shifted_-1                   float64
NPHI_shifted_-1                 float64
PEF_shifted_-1                  float64
DTC_shifted_-1                  float64
SP_shifted_-1                   float64
BS_shifted_-1                   float64
DRHO_shifted_-1                 float64
GROUP_encoded_shifted_-1        float64
FORMATION_encoded_shifted_-1    float64
WELL                             object
DEPTH_MD                        float64
X_LOC_gradient                  float64
Y_LOC_gradient                  float64
